In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
def date_time(data):
    months = {
    "Jan": 1,
    "Feb": 2,
    "Mar": 3,
    "Apr": 4,
    "May": 5,
    "Jun": 6,
    "Jul": 7,
    "Aug": 8,
    "Sep": 9,
    "Oct": 10,
    "Nov": 11,
    "Dec": 12
}
    data["Year"] = [word[8:] for word in data["State"]]
    data["Year"] = data["Year"].astype(int)
    data["Day"] = [word[:2] for word in data["Date"]]
    data["Day"] = data["Day"].astype(int)
    data["Month"] = [months[word[3:6]] for word in data["Date"]]
    data["Hour"] = [time[7:-2] for time in data["Date"]]
    data["Hour"] = data["Hour"].astype(int)
    data["datetime"] = pd.to_datetime(data[["Year", "Month", "Day"]])
    data.drop(labels=["Day", "Month"], axis=1, inplace=True)

def check_holiday(data, holidays):
    holidays["date"] = pd.to_datetime(holidays["date"])
    data["Holidays"] = data["datetime"].isin(holidays["date"])

def convert_date(data):
    data["currentYear"] = data.datetime.dt.year
    data["currentMonth"] = data.datetime.dt.month
    data["currentDay"] = data.datetime.dt.day
    data["currnetDayOfWeek"] = data.datetime.dt.dayofweek
    data["currnetDayOfYear"] = data.datetime.dt.dayofyear

    data.drop(labels=["Date", "Year", "datetime", "State"], axis=1, inplace=True)

def change_data(data, holidays):
    check_holiday(data, holidays)
    convert_date(data)
    

In [3]:
data = pd.read_excel("Data/Electricity/Data 2017-2023 (1).xlsx")
holidays = pd.read_csv("Data/Public Holidays/2018.csv")
weather = pd.read_csv("Data/Weather/weather.csv")
weather = weather[["datetime", "tempmax", "tempmin", "temp", "feelslikemax", "feelslikemin", "feelslike", "solarenergy"]]
data.head(), len(data)

(          State         Date  Hourly Demand Met (in MW)
 0  Delhi - 2023  01-Jan 12am                    2252.78
 1  Delhi - 2023   01-Jan 1am                    2013.40
 2  Delhi - 2023   01-Jan 2am                    1872.10
 3  Delhi - 2023   01-Jan 3am                    1806.61
 4  Delhi - 2023   01-Jan 4am                    1855.44,
 61344)

In [4]:
holidays.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 366 entries, 0 to 365
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   date          366 non-null    object
 1   day           366 non-null    object
 2   holiday       366 non-null    object
 3   holiday_type  366 non-null    object
dtypes: object(4)
memory usage: 11.6+ KB


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61344 entries, 0 to 61343
Data columns (total 3 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   State                      61344 non-null  object 
 1   Date                       61344 non-null  object 
 2   Hourly Demand Met (in MW)  61344 non-null  float64
dtypes: float64(1), object(2)
memory usage: 1.4+ MB


In [6]:
weather["datetime"] = weather["datetime"].astype("datetime64[ns]")
weather.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2921 entries, 0 to 2920
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   datetime      2921 non-null   datetime64[ns]
 1   tempmax       2921 non-null   float64       
 2   tempmin       2921 non-null   float64       
 3   temp          2921 non-null   float64       
 4   feelslikemax  2921 non-null   float64       
 5   feelslikemin  2921 non-null   float64       
 6   feelslike     2921 non-null   float64       
 7   solarenergy   2921 non-null   float64       
dtypes: datetime64[ns](1), float64(7)
memory usage: 182.7 KB


In [7]:
data_copy = data.copy()
data_copy.head()

,State,Date,Hourly Demand Met (in MW)
0,Delhi - 2023,01-Jan 12am,2252.78
1,Delhi - 2023,01-Jan 1am,2013.40
2,Delhi - 2023,01-Jan 2am,1872.10
3,Delhi - 2023,01-Jan 3am,1806.61
4,Delhi - 2023,01-Jan 4am,1855.44


In [8]:
date_time(data_copy)
data_copy.head()

,State,Date,Hourly Demand Met (in MW),Year,Hour,datetime
0,Delhi - 2023,01-Jan 12am,2252.78,2023,12,2023-01-01
1,Delhi - 2023,01-Jan 1am,2013.40,2023,1,2023-01-01
2,Delhi - 2023,01-Jan 2am,1872.10,2023,2,2023-01-01
3,Delhi - 2023,01-Jan 3am,1806.61,2023,3,2023-01-01
4,Delhi - 2023,01-Jan 4am,1855.44,2023,4,2023-01-01


In [9]:
data_copy_copy = data_copy.copy()
holidays_copy = holidays.copy()

In [10]:
holidays_copy = holidays_copy[~holidays_copy['holiday_type'].isin(['observance', 'Restricted', 'Restricted Holiday'])]
holidays_copy.head()

,date,day,holiday,holiday_type
4,2017-01-26,Thursday,Republic Day,Gazetted Holiday
9,2017-02-24,Friday,Maha Shivaratri/Shivaratri,Gazetted Holiday
11,2017-03-13,Monday,Holi,Gazetted Holiday
13,2017-04-04,Tuesday,Rama Navami,Gazetted Holiday
14,2017-04-09,Sunday,Mahavir Jayanti,Gazetted Holiday


In [12]:
result = pd.merge(data_copy, weather, on="datetime", how="left")
result

,State,Date,Hourly Demand Met (in MW),Year,Hour,datetime,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,solarenergy
0,Delhi - 2023,01-Jan 12am,2252.78,2023,12,2023-01-01,20.1,7.9,13.4,20.1,7.9,13.3,12.5
1,Delhi - 2023,01-Jan 1am,2013.40,2023,1,2023-01-01,20.1,7.9,13.4,20.1,7.9,13.3,12.5
2,Delhi - 2023,01-Jan 2am,1872.10,2023,2,2023-01-01,20.1,7.9,13.4,20.1,7.9,13.3,12.5
3,Delhi - 2023,01-Jan 3am,1806.61,2023,3,2023-01-01,20.1,7.9,13.4,20.1,7.9,13.3,12.5
4,Delhi - 2023,01-Jan 4am,1855.44,2023,4,2023-01-01,20.1,7.9,13.4,20.1,7.9,13.3,12.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...
70099,Delhi - 2017,31-Dec 7pm,2986.64,2017,7,2017-12-31,22.3,8.0,13.3,22.3,7.9,13.2,14.1
70100,Delhi - 2017,31-Dec 8pm,2780.17,2017,8,2017-12-31,22.3,8.0,13.3,22.3,7.9,13.2,14.1
70101,Delhi - 2017,31-Dec 9pm,2539.07,2017,9,2017-12-31,22.3,8.0,13.3,22.3,7.9,13.2,14.1
70102,Delhi - 2017,31-Dec 10pm,2316.31,2017,10,2017-12-31,22.3,8.0,13.3,22.3,7.9,13.2,14.1


In [13]:
change_data(result, holidays_copy)

In [14]:
result.head()

,Hourly Demand Met (in MW),Hour,tempmax,tempmin,temp,feelslikemax,feelslikemin,feelslike,solarenergy,Holidays,currentYear,currentMonth,currentDay,currnetDayOfWeek,currnetDayOfYear
0,2252.78,12,20.1,7.9,13.4,20.1,7.9,13.3,12.5,False,2023,1,1,6,1
1,2013.40,1,20.1,7.9,13.4,20.1,7.9,13.3,12.5,False,2023,1,1,6,1
2,1872.10,2,20.1,7.9,13.4,20.1,7.9,13.3,12.5,False,2023,1,1,6,1
3,1806.61,3,20.1,7.9,13.4,20.1,7.9,13.3,12.5,False,2023,1,1,6,1
4,1855.44,4,20.1,7.9,13.4,20.1,7.9,13.3,12.5,False,2023,1,1,6,1


In [15]:
holidays_copy.head()

,date,day,holiday,holiday_type
4,2017-01-26,Thursday,Republic Day,Gazetted Holiday
9,2017-02-24,Friday,Maha Shivaratri/Shivaratri,Gazetted Holiday
11,2017-03-13,Monday,Holi,Gazetted Holiday
13,2017-04-04,Tuesday,Rama Navami,Gazetted Holiday
14,2017-04-09,Sunday,Mahavir Jayanti,Gazetted Holiday


In [16]:
result.value_counts()

Hourly Demand Met (in MW)  Hour  tempmax  tempmin  temp  feelslikemax  feelslikemin  feelslike  solarenergy  Holidays  currentYear  currentMonth  currentDay  currnetDayOfWeek  currnetDayOfYear
527.28                     3     23.7     10.2     16.2  23.7          10.2          16.2       14.3         False     2018         12            7           4                 341                 2
2895.03                    7     28.1     18.5     23.1  27.9          18.5          23.0       19.8         False     2018         2             25          6                 56                  2
2892.05                    8     27.0     17.0     21.3  27.3          17.0          21.3       15.0         False     2018         11            15          3                 319                 2
4696.59                    8     35.4     24.9     29.9  40.6          24.9          33.1       22.1         False     2018         9             19          2                 262                 2
2892.66        

In [19]:
result.to_csv("trainingset.csv")